In [5]:
from pathlib import Path
from src.models import ConvLSTMSplineModel, CubicSplineLayer
from src.losses import SplineLoss
from src.dataset import DatasetGenerator, percentile_contrast, scale_to_0_255
import keras
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
DATASET_PATH = Path("/home/me/workspace/probformer/data/greenearthnet/train/29SND")

SPLINE_PATH = Path("/home/me/workspace/bspline_ndvi")

LOGS_PATH = SPLINE_PATH / "logs"

CHECKPOINTS_PATH = SPLINE_PATH / "checkpoints"
BEST_CHECKPOINT_PATH = CHECKPOINTS_PATH / "best_model.keras"

: 

In [ ]:

def customize_axes(fig):
    fig.tight_layout()
    for i, ax in enumerate(fig.axes):
        ax.tick_params(
            bottom=False,
            left=False,
            labelbottom=False, 
            labelleft=False
        )

    plt.subplots_adjust(wspace=0, hspace=0)


def visualize_model():
    model = keras.models.load_model(BEST_CHECKPOINT_PATH)
    if not isinstance(model, ConvLSTMSplineModel):
        raise ValueError("Model is not instance of ConvLSTMSplineModel")
    
    dataset = DatasetGenerator(DATASET_PATH).get_dataset()

    # fig, axes = plt.subplots(3, 2, figsize=(5,8))
    for x, y in dataset.batch(1).shuffle(10).take(1):
        
        y_pred = model(x)
        y_pred = y_pred["deltas"].numpy()
        y_true = y["deltas"].numpy()
        
        plot_times = range(5)
        fig, axes = plt.subplots(len(plot_times), 2, figsize=(9, 9))
        
        for i in plot_times:
            for band in range(4):
                print("Band", band)
                print("Y True: ", np.mean(y_true[0, i, :, :, band]))
                print("Y Pred: ", np.mean(y_pred[0, i, :, :, band]))

            axes[i][0].imshow(y_true[0, i, :, :, 0:3], aspect="auto")
            axes[i][1].imshow(y_pred[0, i, :, :, 0:3], aspect="auto")
        
        customize_axes(fig)
        
    plt.show()

visualize_model()